# Import Library

In [63]:
from tqdm import tqdm
import time
import joblib
import warnings
import os

import numpy as np
import pandas as pd

from catboost import CatBoostRegressor

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

%reload_ext autotime

time: 0 ns (started: 2024-08-01 03:59:20 +09:00)


# Load Data

In [64]:
# 기본 경로 설정
base_path = './data/bami/total/'

# 파일명 리스트
file_names = ['방문지정보.csv', '여행.csv', '여행객 Master.csv']

# 데이터프레임 딕셔너리에 로드
dataframes = {file_name: pd.read_csv(os.path.join(base_path, file_name)) for file_name in file_names}

# 각각의 데이터프레임 할당
visit_area_info = dataframes['방문지정보.csv']
travel = dataframes['여행.csv']
traveler_master = dataframes['여행객 Master.csv']

time: 313 ms (started: 2024-08-01 03:59:20 +09:00)


# Preprocessing

## visit_area_info 방문지 정보

In [65]:
# 관광지 선택
# 자연관광지, 역사/유적/종교 시설(문화재, 박물관, 촬영지, 절 등), 문화 시설(공연장, 영화관, 전시관 등), 상업지구(거리, 시장, 쇼핑시설)
# 레저/스포츠 관련 시설(스키, 카트, 수상레저), 테마시설(놀이공원, 워터파크), 산책로, 둘레길 등, 지역 축제/행사
visit_area_info = visit_area_info.loc[visit_area_info['VISIT_AREA_TYPE_CD'].between(1, 8)]

# LOTNO_ADDR (지번주소) 열에 결측값이 있는 행 제거 후 인덱스 재설정
# ex) 경기 수원시 팔달구 매향동 3-32
visit_area_info = visit_area_info.dropna(subset=['LOTNO_ADDR']).reset_index(drop=True)

# 데이터 확인
visit_area_info.head()

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,ROAD_NM_CD,LOTNO_CD,POI_ID,POI_NM,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,region
0,2304300002,e_e000004,2,화성 관광열차 안내소 연무대 매표소,2023-04-30,2023-04-30,경기 수원시 팔달구 창룡대로103번길 20,경기 수원시 팔달구 매향동 3-32,127.023339,37.287878,NaN,NaN,POI01000000ALZU7R,동대문종합시장 악세서리부자재시장,60.0,2,N,10.0,NaN,4.0,3.0,4.0,NaN,capital
1,2304300003,e_e000004,3,창룡문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 남수동,127.025143,37.287791,NaN,NaN,POI010000006N1USC,창룡문,30.0,2,N,1.0,NaN,4.0,4.0,4.0,NaN,capital
2,2304300004,e_e000004,4,수원 화성 화홍문,2023-04-30,2023-04-30,NaN,경기 수원시 팔달구 북수동 9000-1,127.017626,37.287546,NaN,NaN,POI01000TR021821V,수원화성 화홍문,60.0,2,N,10.0,NaN,4.0,3.0,3.0,NaN,capital
3,2304300004,e_e000006,4,경춘선 자전거길,2023-04-30,2023-04-30,NaN,경기 가평군 청평면 하천리 158-2,127.436200,37.745958,NaN,NaN,POI01000TR008470V,경춘선자전거길,150.0,1,Y,4.0,NaN,5.0,5.0,5.0,NaN,capital
4,2304290002,e_e000009,2,농협안성팜랜드,2023-04-29,2023-04-29,경기 안성시 공도읍 대신두길 28,경기 안성시 공도읍 신두리 451,127.193517,36.991317,NaN,NaN,POI01000000E66UWC,안성팜랜드,30.0,6,N,1.0,NaN,4.0,4.0,4.0,NaN,capital


time: 31 ms (started: 2024-08-01 03:59:21 +09:00)


In [66]:
# 시도/군구 변수 생성
visit_area_info['SIDO'] = visit_area_info['LOTNO_ADDR'].apply(lambda x: x.split(' ')[0])
visit_area_info['GUNGU'] = visit_area_info['LOTNO_ADDR'].apply(lambda x: x.split(' ')[1])

time: 15 ms (started: 2024-08-01 03:59:21 +09:00)


In [67]:
visit_area_info['SIDO'].value_counts()

SIDO
경기         3232
서울         2328
경북         2223
전북         1719
충남         1688
강원특별자치도    1662
전남         1660
부산         1552
강원         1178
대전         1143
충북         1124
경남         1028
인천          571
대구          441
광주          295
울산          273
세종특별자치시     218
강원도           5
경상북도          4
전라북도          3
경기도           3
부산광역시         3
충청남도          3
전라남도          2
충청북도          2
서울특별시         1
경상남도          1
Name: count, dtype: int64

time: 16 ms (started: 2024-08-01 03:59:21 +09:00)


변수 선택
- TRAVEL_ID: 여행 ID
- VISIT_AREA_NM: 방문 장소 이름
- SIDO: 시/도
- GUNGU: 군/구
- VISIT_AREA_TYPE_CD: 관광 장소 유형
- DGSTFN: 만족도
- REVISIT_INTENTION: 재방문의향
- RCMDTN_INTENTION: 추천의향
- RESIDENCE_TIME_MIN: 체류시간분
- REVISIT_YN: 재방문여부

In [68]:
visit_area_info = visit_area_info[['TRAVEL_ID', 'VISIT_AREA_NM', 'SIDO', 'GUNGU', 'VISIT_AREA_TYPE_CD', 'DGSTFN',
                                   'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'RESIDENCE_TIME_MIN', 'REVISIT_YN']]

visit_area_info.head()

,TRAVEL_ID,VISIT_AREA_NM,SIDO,GUNGU,VISIT_AREA_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,RESIDENCE_TIME_MIN,REVISIT_YN
0,e_e000004,화성 관광열차 안내소 연무대 매표소,경기,수원시,2,4.0,3.0,4.0,60.0,N
1,e_e000004,창룡문,경기,수원시,2,4.0,4.0,4.0,30.0,N
2,e_e000004,수원 화성 화홍문,경기,수원시,2,4.0,3.0,3.0,60.0,N
3,e_e000006,경춘선 자전거길,경기,가평군,1,5.0,5.0,5.0,150.0,Y
4,e_e000009,농협안성팜랜드,경기,안성시,6,4.0,4.0,4.0,30.0,N


time: 15 ms (started: 2024-08-01 03:59:21 +09:00)
